### Replicamos los recursos locales en el entorno virtual

In [ ]:
!wget --no-cache -O init.py -q https://raw.githubusercontent.com/rramosp/ai4eng.v1/main/content/init.py
import init; init.init(force_download=False); init.get_weblink()

replicating local resources


### Descargamos los datos de la competencia al entorno de trabajo

In [ ]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = '.'
!chmod 600 ./kaggle.json
!kaggle competitions download -c udea-ai-4-eng-20251-pruebas-saber-pro-colombia

  0% 0.00/29.9M [00:00<?, ?B/s]
100% 29.9M/29.9M [00:00<00:00, 1.10GB/s]


### Descomprimimos el archivo

In [ ]:
!unzip udea*.zip > /dev/null

In [ ]:
!wc *.csv

   296787    296787   4716673 submission_example.csv
   296787   4565553  59185250 test.csv
   692501  10666231 143732449 train.csv
  1286075  15528571 207634372 total


## Importe de librerías necesarias

In [ ]:
!pip install fuzzywuzzy
!pip install unidecode

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.8/235.8 kB 12.0 MB/s eta 0:00:00


In [ ]:
import unidecode as ud
import pandas as pd
import numpy as np
from fuzzywuzzy import fuzz, process
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder

/usr/local/lib/python3.11/dist-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


### Cargamos el csv como un dataframe de Pandas
Se guarda en la variable train

In [ ]:
train = pd.read_csv("train.csv")
print ("shape of loaded dataframe", train.shape)

shape of loaded dataframe (692500, 21)


In [ ]:
train.columns

Index(['ID', 'PERIODO', 'ESTU_PRGM_ACADEMICO', 'ESTU_PRGM_DEPARTAMENTO',
       'ESTU_VALORMATRICULAUNIVERSIDAD', 'ESTU_HORASSEMANATRABAJA',
       'FAMI_ESTRATOVIVIENDA', 'FAMI_TIENEINTERNET', 'FAMI_EDUCACIONPADRE',
       'FAMI_TIENELAVADORA', 'FAMI_TIENEAUTOMOVIL', 'ESTU_PRIVADO_LIBERTAD',
       'ESTU_PAGOMATRICULAPROPIO', 'FAMI_TIENECOMPUTADOR',
       'FAMI_TIENEINTERNET.1', 'FAMI_EDUCACIONMADRE', 'RENDIMIENTO_GLOBAL',
       'coef_1', 'coef_2', 'coef_3', 'coef_4'],
      dtype='object')

In [ ]:
train.head()

,ID,PERIODO,ESTU_PRGM_ACADEMICO,ESTU_PRGM_DEPARTAMENTO,ESTU_VALORMATRICULAUNIVERSIDAD,ESTU_HORASSEMANATRABAJA,FAMI_ESTRATOVIVIENDA,FAMI_TIENEINTERNET,FAMI_EDUCACIONPADRE,FAMI_TIENELAVADORA,...,ESTU_PRIVADO_LIBERTAD,ESTU_PAGOMATRICULAPROPIO,FAMI_TIENECOMPUTADOR,FAMI_TIENEINTERNET.1,FAMI_EDUCACIONMADRE,RENDIMIENTO_GLOBAL,coef_1,coef_2,coef_3,coef_4
0,904256,20212,ENFERMERIA,BOGOTÁ,Entre 5.5 millones y menos de 7 millones,Menos de 10 horas,Estrato 3,Si,Técnica o tecnológica incompleta,Si,...,N,No,Si,Si,Postgrado,medio-alto,0.322,0.208,0.310,0.267
1,645256,20212,DERECHO,ATLANTICO,Entre 2.5 millones y menos de 4 millones,0,Estrato 3,No,Técnica o tecnológica completa,Si,...,N,No,Si,No,Técnica o tecnológica incompleta,bajo,0.311,0.215,0.292,0.264
2,308367,20203,MERCADEO Y PUBLICIDAD,BOGOTÁ,Entre 2.5 millones y menos de 4 millones,Más de 30 horas,Estrato 3,Si,Secundaria (Bachillerato) completa,Si,...,N,No,No,Si,Secundaria (Bachillerato) completa,bajo,0.297,0.214,0.305,0.264
3,470353,20195,ADMINISTRACION DE EMPRESAS,SANTANDER,Entre 4 millones y menos de 5.5 millones,0,Estrato 4,Si,No sabe,Si,...,N,No,Si,Si,Secundaria (Bachillerato) completa,alto,0.485,0.172,0.252,0.190
4,989032,20212,PSICOLOGIA,ANTIOQUIA,Entre 2.5 millones y menos de 4 millones,Entre 21 y 30 horas,Estrato 3,Si,Primaria completa,Si,...,N,No,Si,Si,Primaria completa,medio-bajo,0.316,0.232,0.285,0.294


## One_hot encoding para la columna 'ESTU_PRGMACADEMICO'

In [ ]:
carreras_unicas = train['ESTU_PRGM_ACADEMICO'].unique()

In [ ]:
train['ESTU_PRGM_ACADEMICO'] = train['ESTU_PRGM_ACADEMICO'].apply(lambda x: ud.unidecode(x.lower()))

In [ ]:
train.head()

,ID,PERIODO,ESTU_PRGM_ACADEMICO,ESTU_PRGM_DEPARTAMENTO,ESTU_VALORMATRICULAUNIVERSIDAD,ESTU_HORASSEMANATRABAJA,FAMI_ESTRATOVIVIENDA,FAMI_TIENEINTERNET,FAMI_EDUCACIONPADRE,FAMI_TIENELAVADORA,...,ESTU_PRIVADO_LIBERTAD,ESTU_PAGOMATRICULAPROPIO,FAMI_TIENECOMPUTADOR,FAMI_TIENEINTERNET.1,FAMI_EDUCACIONMADRE,RENDIMIENTO_GLOBAL,coef_1,coef_2,coef_3,coef_4
0,904256,20212,enfermeria,BOGOTÁ,Entre 5.5 millones y menos de 7 millones,Menos de 10 horas,Estrato 3,Si,Técnica o tecnológica incompleta,Si,...,N,No,Si,Si,Postgrado,medio-alto,0.322,0.208,0.310,0.267
1,645256,20212,derecho,ATLANTICO,Entre 2.5 millones y menos de 4 millones,0,Estrato 3,No,Técnica o tecnológica completa,Si,...,N,No,Si,No,Técnica o tecnológica incompleta,bajo,0.311,0.215,0.292,0.264
2,308367,20203,mercadeo y publicidad,BOGOTÁ,Entre 2.5 millones y menos de 4 millones,Más de 30 horas,Estrato 3,Si,Secundaria (Bachillerato) completa,Si,...,N,No,No,Si,Secundaria (Bachillerato) completa,bajo,0.297,0.214,0.305,0.264
3,470353,20195,administracion de empresas,SANTANDER,Entre 4 millones y menos de 5.5 millones,0,Estrato 4,Si,No sabe,Si,...,N,No,Si,Si,Secundaria (Bachillerato) completa,alto,0.485,0.172,0.252,0.190
4,989032,20212,psicologia,ANTIOQUIA,Entre 2.5 millones y menos de 4 millones,Entre 21 y 30 horas,Estrato 3,Si,Primaria completa,Si,...,N,No,Si,Si,Primaria completa,medio-bajo,0.316,0.232,0.285,0.294


In [ ]:
counts = train['ESTU_PRGM_ACADEMICO'].value_counts().to_frame()
prgm_acads = (counts.loc[counts['count'] > 6000]).index
prgm_acads
len(prgm_acads)

20

In [ ]:
array = [element for element in train['ESTU_PRGM_ACADEMICO'] if element not in prgm_acads ]
array

condition = train['ESTU_PRGM_ACADEMICO'].isin(array)
train.loc[condition, 'ESTU_PRGM_ACADEMICO'] = "otros"
train.head()

,ID,PERIODO,ESTU_PRGM_ACADEMICO,ESTU_PRGM_DEPARTAMENTO,ESTU_VALORMATRICULAUNIVERSIDAD,ESTU_HORASSEMANATRABAJA,FAMI_ESTRATOVIVIENDA,FAMI_TIENEINTERNET,FAMI_EDUCACIONPADRE,FAMI_TIENELAVADORA,...,ESTU_PRIVADO_LIBERTAD,ESTU_PAGOMATRICULAPROPIO,FAMI_TIENECOMPUTADOR,FAMI_TIENEINTERNET.1,FAMI_EDUCACIONMADRE,RENDIMIENTO_GLOBAL,coef_1,coef_2,coef_3,coef_4
0,904256,20212,enfermeria,BOGOTÁ,Entre 5.5 millones y menos de 7 millones,Menos de 10 horas,Estrato 3,Si,Técnica o tecnológica incompleta,Si,...,N,No,Si,Si,Postgrado,medio-alto,0.322,0.208,0.310,0.267
1,645256,20212,derecho,ATLANTICO,Entre 2.5 millones y menos de 4 millones,0,Estrato 3,No,Técnica o tecnológica completa,Si,...,N,No,Si,No,Técnica o tecnológica incompleta,bajo,0.311,0.215,0.292,0.264
2,308367,20203,otros,BOGOTÁ,Entre 2.5 millones y menos de 4 millones,Más de 30 horas,Estrato 3,Si,Secundaria (Bachillerato) completa,Si,...,N,No,No,Si,Secundaria (Bachillerato) completa,bajo,0.297,0.214,0.305,0.264
3,470353,20195,administracion de empresas,SANTANDER,Entre 4 millones y menos de 5.5 millones,0,Estrato 4,Si,No sabe,Si,...,N,No,Si,Si,Secundaria (Bachillerato) completa,alto,0.485,0.172,0.252,0.190
4,989032,20212,psicologia,ANTIOQUIA,Entre 2.5 millones y menos de 4 millones,Entre 21 y 30 horas,Estrato 3,Si,Primaria completa,Si,...,N,No,Si,Si,Primaria completa,medio-bajo,0.316,0.232,0.285,0.294


In [ ]:
train['ESTU_PRGM_ACADEMICO'].value_counts()

,count
ESTU_PRGM_ACADEMICO,
otros,251299
administracion de empresas,76679
derecho,53244
contaduria publica,51846
psicologia,49309
ingenieria industrial,34794
ingenieria civil,20943
medicina,18099
ingenieria de sistemas,18028


In [ ]:
encode = OneHotEncoder()

codificadorPrg = encode.fit_transform(train[['ESTU_PRGM_ACADEMICO']])

arreglo = pd.DataFrame(codificadorPrg.toarray(), columns = encode.categories_)
print(arreglo)
train.drop('ESTU_PRGM_ACADEMICO', axis = 1)

train = pd.concat([train, arreglo], axis = 'columns')
train['ESTU_PRGM_ACADEMICO']

       administracion de empresas administracion en salud ocupacional  \
0                             0.0                                 0.0   
1                             0.0                                 0.0   
2                             0.0                                 0.0   
3                             1.0                                 0.0   
4                             0.0                                 0.0   
...                           ...                                 ...   
692495                        0.0                                 0.0   
692496                        0.0                                 0.0   
692497                        0.0                                 1.0   
692498                        0.0                                 0.0   
692499                        0.0                                 0.0   

       administracion financiera arquitectura comunicacion social  \
0                            0.0          0.0         

,ESTU_PRGM_ACADEMICO
0,enfermeria
1,derecho
2,otros
3,administracion de empresas
4,psicologia
...,...
692495,otros
692496,psicologia
692497,administracion en salud ocupacional
692498,psicologia


In [ ]:
train.head()

,ID,PERIODO,ESTU_PRGM_ACADEMICO,ESTU_PRGM_DEPARTAMENTO,ESTU_VALORMATRICULAUNIVERSIDAD,ESTU_HORASSEMANATRABAJA,FAMI_ESTRATOVIVIENDA,FAMI_TIENEINTERNET,FAMI_EDUCACIONPADRE,FAMI_TIENELAVADORA,...,"(ingenieria de sistemas,)","(ingenieria electronica,)","(ingenieria industrial,)","(ingenieria mecanica,)","(licenciatura en pedagogia infantil,)","(medicina,)","(negocios internacionales,)","(otros,)","(psicologia,)","(trabajo social,)"
0,904256,20212,enfermeria,BOGOTÁ,Entre 5.5 millones y menos de 7 millones,Menos de 10 horas,Estrato 3,Si,Técnica o tecnológica incompleta,Si,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,645256,20212,derecho,ATLANTICO,Entre 2.5 millones y menos de 4 millones,0,Estrato 3,No,Técnica o tecnológica completa,Si,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,308367,20203,otros,BOGOTÁ,Entre 2.5 millones y menos de 4 millones,Más de 30 horas,Estrato 3,Si,Secundaria (Bachillerato) completa,Si,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,470353,20195,administracion de empresas,SANTANDER,Entre 4 millones y menos de 5.5 millones,0,Estrato 4,Si,No sabe,Si,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,989032,20212,psicologia,ANTIOQUIA,Entre 2.5 millones y menos de 4 millones,Entre 21 y 30 horas,Estrato 3,Si,Primaria completa,Si,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


## Categorización ordinal de las columnas categóricas ordinales

In [ ]:
train['RENDIMIENTO_GLOBAL'].head()

,RENDIMIENTO_GLOBAL
0,medio-alto
1,bajo
2,bajo
3,alto
4,medio-bajo


In [ ]:
train.columns

Index([                                    'ID',
                                      'PERIODO',
                          'ESTU_PRGM_ACADEMICO',
                       'ESTU_PRGM_DEPARTAMENTO',
               'ESTU_VALORMATRICULAUNIVERSIDAD',
                      'ESTU_HORASSEMANATRABAJA',
                         'FAMI_ESTRATOVIVIENDA',
                           'FAMI_TIENEINTERNET',
                          'FAMI_EDUCACIONPADRE',
                           'FAMI_TIENELAVADORA',
                          'FAMI_TIENEAUTOMOVIL',
                        'ESTU_PRIVADO_LIBERTAD',
                     'ESTU_PAGOMATRICULAPROPIO',
                         'FAMI_TIENECOMPUTADOR',
                         'FAMI_TIENEINTERNET.1',
                          'FAMI_EDUCACIONMADRE',
                           'RENDIMIENTO_GLOBAL',
                                       'coef_1',
                                       'coef_2',
                                       'coef_3',
                    

## Quitarle los valores nulos a todas las columnas categóricas ordinales


### Conteo de valores nulos

In [ ]:
#contar la cantidad de veces que aparece cada categoría o valor en la columna
#'FAMI_EDUCACIONMADRE' Para esta variable la idea es conocer los valores nulos de la variable.
train['FAMI_EDUCACIONMADRE'].value_counts()

,count
FAMI_EDUCACIONMADRE,
Secundaria (Bachillerato) completa,141744
Primaria incompleta,99420
Técnica o tecnológica completa,89542
Educación profesional completa,85326
Secundaria (Bachillerato) incompleta,81012
Primaria completa,56125
Postgrado,46246
Técnica o tecnológica incompleta,27533
Educación profesional incompleta,22470


In [ ]:
print(f"Cantidad nulos valor matrícula = {train['ESTU_VALORMATRICULAUNIVERSIDAD'].isna().sum()}")
print(f"Cantidad nulos horas semanales trabaja = {train['ESTU_HORASSEMANATRABAJA'].isna().sum()}")
print(f"Cantidad nulos estrato vivienda = {train['FAMI_ESTRATOVIVIENDA'].isna().sum()}")
print(f"Cantidad nulos educación padre = {train['FAMI_EDUCACIONPADRE'].isna().sum()}")
print(f"Cantidad nulos educación madre = {train['FAMI_EDUCACIONMADRE'].isna().sum()}")
print(f"Cantidad nulos rendimiento global = {train['RENDIMIENTO_GLOBAL'].isna().sum()}")

Cantidad nulos valor matrícula = 6287
Cantidad nulos horas semanales trabaja = 30857
Cantidad nulos estrato vivienda = 32137
Cantidad nulos educación padre = 23178
Cantidad nulos educación madre = 23664
Cantidad nulos rendimiento global = 0


### Cambio de valores nulos por 'no info'

In [ ]:
train['ESTU_VALORMATRICULAUNIVERSIDAD'] = train['ESTU_VALORMATRICULAUNIVERSIDAD'].fillna("no info")
train['ESTU_HORASSEMANATRABAJA'] = train['ESTU_HORASSEMANATRABAJA'].fillna("no info")
train['FAMI_ESTRATOVIVIENDA'] = train['FAMI_ESTRATOVIVIENDA'].fillna("no info")
train['FAMI_EDUCACIONPADRE'] = train['FAMI_EDUCACIONPADRE'].fillna("no info")
train['FAMI_EDUCACIONMADRE'] = train['FAMI_EDUCACIONMADRE'].fillna("no info")
train['RENDIMIENTO_GLOBAL'] = train['RENDIMIENTO_GLOBAL'].fillna("no info")

## Codificación de valores categóricos ordinales.

está utilizando OrdinalEncoder de sklearn.preprocessing para convertir variables categóricas ordinales en valores numéricos según un orden predefinido.


In [ ]:
categories = [
    ['no info', 'No pagó matrícula', 'Menos de 500 mil', 'Entre 500 mil y menos de 1 millón',
     'Entre 1 millón y menos de 2.5 millones', 'Entre 2.5 millones y menos de 4 millones',
     'Entre 4 millones y menos de 5.5 millones', 'Entre 5.5 millones y menos de 7 millones',
     'Más de 7 millones'],  # ESTU_VALORMATRICULAUNIVERSIDAD
    ['no info', '0', 'Menos de 10 horas', 'Entre 11 y 20 horas', 'Entre 21 y 30 horas', 'Más de 30 horas'],  # ESTU_HORASSEMANATRABAJA
    ['no info', 'Sin Estrato', 'Estrato 1', 'Estrato 2', 'Estrato 3', 'Estrato 4', 'Estrato 5', 'Estrato 6'],  # FAMI_ESTRATOVIVIENDA
    ['no info', 'Ninguno', 'No sabe', 'No Aplica', 'Primaria incompleta', 'Primaria completa',
     'Secundaria (Bachillerato) incompleta', 'Secundaria (Bachillerato) completa', 'Técnica o tecnológica incompleta',
     'Técnica o tecnológica completa', 'Educación profesional incompleta', 'Educación profesional completa', 'Postgrado'],  # FAMI_EDUCACIONPADRE
    ['no info', 'Ninguno', 'No sabe', 'No Aplica', 'Primaria incompleta', 'Primaria completa',
     'Secundaria (Bachillerato) incompleta', 'Secundaria (Bachillerato) completa', 'Técnica o tecnológica incompleta',
     'Técnica o tecnológica completa', 'Educación profesional incompleta', 'Educación profesional completa', 'Postgrado'],  # FAMI_EDUCACIONMADRE
    ['no info', 'bajo', 'medio-bajo', 'medio-alto', 'alto']  # RENDIMIENTO_GLOBAL
]



In [ ]:
codificador = OrdinalEncoder(categories=categories)
train[['ESTU_VALORMATRICULAUNIVERSIDAD', 'ESTU_HORASSEMANATRABAJA',
       'FAMI_ESTRATOVIVIENDA', 'FAMI_EDUCACIONPADRE',
       'FAMI_EDUCACIONMADRE', 'RENDIMIENTO_GLOBAL']] = codificador.fit_transform(
    train[['ESTU_VALORMATRICULAUNIVERSIDAD', 'ESTU_HORASSEMANATRABAJA',
           'FAMI_ESTRATOVIVIENDA', 'FAMI_EDUCACIONPADRE',
           'FAMI_EDUCACIONMADRE', 'RENDIMIENTO_GLOBAL']]
       )

In [ ]:
train.head()

,ID,PERIODO,ESTU_PRGM_ACADEMICO,ESTU_PRGM_DEPARTAMENTO,ESTU_VALORMATRICULAUNIVERSIDAD,ESTU_HORASSEMANATRABAJA,FAMI_ESTRATOVIVIENDA,FAMI_TIENEINTERNET,FAMI_EDUCACIONPADRE,FAMI_TIENELAVADORA,...,"(ingenieria de sistemas,)","(ingenieria electronica,)","(ingenieria industrial,)","(ingenieria mecanica,)","(licenciatura en pedagogia infantil,)","(medicina,)","(negocios internacionales,)","(otros,)","(psicologia,)","(trabajo social,)"
0,904256,20212,enfermeria,BOGOTÁ,7.0,2.0,4.0,Si,8.0,Si,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,645256,20212,derecho,ATLANTICO,5.0,1.0,4.0,No,9.0,Si,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,308367,20203,otros,BOGOTÁ,5.0,5.0,4.0,Si,7.0,Si,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
3,470353,20195,administracion de empresas,SANTANDER,6.0,1.0,5.0,Si,2.0,Si,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,989032,20212,psicologia,ANTIOQUIA,5.0,4.0,4.0,Si,5.0,Si,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [ ]:
# Reemplaza los valores nulos (NaN) por la cadena 'no info' en esa columna.
train['ESTU_PAGOMATRICULAPROPIO'] = train['ESTU_PAGOMATRICULAPROPIO'].fillna('no info')
#Muestra cuántas veces aparece cada valor único en la columna (incluido 'no info').
train['ESTU_PAGOMATRICULAPROPIO'].value_counts()

,count
ESTU_PAGOMATRICULAPROPIO,
No,382201
Si,303801
no info,6498


In [ ]:
#Elimina del DataFrame todas las filas donde 'ESTU_PAGOMATRICULAPROPIO' tenga el valor 'no info'.
train = train[train['ESTU_PAGOMATRICULAPROPIO'] != 'no info']

In [ ]:
#Vuelve a mostrar los conteos, ahora sin las filas que tenían originalmente valores nulos.
train['ESTU_PAGOMATRICULAPROPIO'].value_counts()

,count
ESTU_PAGOMATRICULAPROPIO,
No,382201
Si,303801


In [ ]:
#Esto imprime cuántos valores nulos (NaN) hay en la columna 'ESTU_PAGOMATRICULAPROPIO'.
print(f"Cantidad nulos tiene internet = {train['ESTU_PAGOMATRICULAPROPIO'].isna().sum()}")
train['FAMI_TIENEINTERNET'].value_counts()

Cantidad nulos tiene internet = 0


,count
FAMI_TIENEINTERNET,
Si,588047
No,73033


## Conversión One-Hot de las columnas FAMI_TIENEINTERNET y ESTU_PAGOMATRICULAPROPIO

In [ ]:
one_hot_encoded = pd.get_dummies(train['ESTU_PAGOMATRICULAPROPIO'], prefix = 'ESTU_PAGOMATRICULAPROPIO')
one_hot_encoded

,ESTU_PAGOMATRICULAPROPIO_No,ESTU_PAGOMATRICULAPROPIO_Si
0,True,False
1,True,False
2,True,False
3,True,False
4,True,False
...,...,...
692495,False,True
692496,True,False
692497,False,True
692498,False,True


In [ ]:
train = pd.concat([train, one_hot_encoded], axis = 1)
train = train.drop('ESTU_PAGOMATRICULAPROPIO', axis = 1)

In [ ]:
train['ESTU_PAGOMATRICULAPROPIO_No'].head()

,ESTU_PAGOMATRICULAPROPIO_No
0,True
1,True
2,True
3,True
4,True


In [ ]:
print(train.columns)

Index([                                    'ID',
                                      'PERIODO',
                          'ESTU_PRGM_ACADEMICO',
                       'ESTU_PRGM_DEPARTAMENTO',
               'ESTU_VALORMATRICULAUNIVERSIDAD',
                      'ESTU_HORASSEMANATRABAJA',
                         'FAMI_ESTRATOVIVIENDA',
                           'FAMI_TIENEINTERNET',
                          'FAMI_EDUCACIONPADRE',
                           'FAMI_TIENELAVADORA',
                          'FAMI_TIENEAUTOMOVIL',
                        'ESTU_PRIVADO_LIBERTAD',
                         'FAMI_TIENECOMPUTADOR',
                         'FAMI_TIENEINTERNET.1',
                          'FAMI_EDUCACIONMADRE',
                           'RENDIMIENTO_GLOBAL',
                                       'coef_1',
                                       'coef_2',
                                       'coef_3',
                                       'coef_4',
                ('ad

In [ ]:
one_hot_encoded = pd.get_dummies(train['FAMI_TIENEINTERNET'], prefix = 'FAMI_TIENEINTERNET')
one_hot_encoded

,FAMI_TIENEINTERNET_No,FAMI_TIENEINTERNET_Si
0,False,True
1,True,False
2,False,True
3,False,True
4,False,True
...,...,...
692495,False,True
692496,False,True
692497,False,True
692498,True,False


In [ ]:
train = pd.concat([train, one_hot_encoded], axis = 1)
train = train.drop('FAMI_TIENEINTERNET', axis = 1)

In [ ]:
train['FAMI_TIENEINTERNET_Si'].head()

,FAMI_TIENEINTERNET_Si
0,True
1,False
2,True
3,True
4,True


In [ ]:
train.columns

Index([                                    'ID',
                                      'PERIODO',
                          'ESTU_PRGM_ACADEMICO',
                       'ESTU_PRGM_DEPARTAMENTO',
               'ESTU_VALORMATRICULAUNIVERSIDAD',
                      'ESTU_HORASSEMANATRABAJA',
                         'FAMI_ESTRATOVIVIENDA',
                          'FAMI_EDUCACIONPADRE',
                           'FAMI_TIENELAVADORA',
                          'FAMI_TIENEAUTOMOVIL',
                        'ESTU_PRIVADO_LIBERTAD',
                         'FAMI_TIENECOMPUTADOR',
                         'FAMI_TIENEINTERNET.1',
                          'FAMI_EDUCACIONMADRE',
                           'RENDIMIENTO_GLOBAL',
                                       'coef_1',
                                       'coef_2',
                                       'coef_3',
                                       'coef_4',
                ('administracion de empresas',),
       ('administrac

In [ ]:
train.to_csv('train1.csv', index = True)